In [1]:
import ROOT
import stage0
import utils

%jsroot on

Welcome to JupyROOT 6.28/00


## Load Config
The config holds all the information on how to construct the histograms and other general settings

In [2]:
# # config = toml.load("stage0_config.toml")
# config = config_loader.load("../configs/stage0_config.toml")
# # config

## Read Calibration
Read the detector calibration from csv file

In [5]:
# import csv
# import numpy as np

# def extract_calib_from_csv(filename, numch):
#     # Initialize the arrays with default values
#     calib_slope = np.ones(numch)
#     calib_offset = np.zeros(numch)

#     with open(filename, 'r') as file:
#         reader = csv.reader(file)
#         next(reader)  # Skip the header
        
#         for row in reader:
#             index = int(row[0])
#             calib_slope[index] = float(row[1])
#             calib_offset[index] = float(row[2])
            
#     return calib_slope, calib_offset

# # Usage
# filename = "../calib/calib_apr.csv"
# numch = config['general']['numch']
# calib_slope, calib_offset = extract_calib_from_csv(filename, numch)



## NUMBA Function
Create JIT compiled C++ function for creating hEgam column

In [6]:
# @ROOT.Numba.Declare(["int","int"], "double")
# def calc_Egam(detector,PulseHeight):
#     return PulseHeight*calib_slope[detector]+calib_offset[detector]

## Create RDataFrame
Create data frame and define new columns then filter by channel. Create histogram models and then fill histograms.

In [ ]:
# ROOT.EnableImplicitMT()

# tree_name = config['general']['tree_name']
# numch = config['general']['numch']
# fp_length = config['general']['fp_length']
# pu_flag = config['general']['pu_flag']
# fo_flag = config['general']['fo_flag']

# # file_name = "data/rawroot_run_0064*"
# file_name = "data/rawroot_run_0014*"


# df = ROOT.RDataFrame(tree_name, file_name)


# s1 =  "PulseHeight != 0"
# s2 =  "PulseHeight == 0"
# s3 =  f"PulseHeight == 0 && Flags == {pu_flag}"
# s4 =  f"PulseHeight == 0 && Flags != {pu_flag}"
# s5 =  f"PulseHeight != 0 && Flags == {pu_flag}"
# s6 =  f"PulseHeight != 0 && Flags != {pu_flag}"
# s7 =  f"Flags == {pu_flag}"
# s8 =  f"Flags != {pu_flag}"

# df = df.Filter(f" detector == {1}")
# df1 = df.Filter(f" detector == {1} &&"+s1)
# df2 = df.Filter(f" detector == {1} &&"+s2)
# df3 = df.Filter(f" detector == {1} &&"+s3)
# df4 = df.Filter(f" detector == {1} &&"+s4)
# df5 = df.Filter(f" detector == {1} &&"+s5)
# df6 = df.Filter(f" detector == {1} &&"+s6)
# df7 = df.Filter(f" detector == {1} &&"+s7)
# df8 = df.Filter(f" detector == {1} &&"+s8)

# c = df.Count().GetValue()
# c1 = df1.Count().GetValue()
# c2 = df2.Count().GetValue()
# c3 = df3.Count().GetValue()
# c4 = df4.Count().GetValue()
# c5 = df5.Count().GetValue()
# c6 = df6.Count().GetValue()
# c7 = df7.Count().GetValue()
# c8 = df8.Count().GetValue()

# print("no cuts:" + f" {c} counts")
# print(s1 + f": {c1} counts {c1/c*100}%")
# print(s2 + f": {c2} counts {c2/c*100}%")
# print(s7 + f": {c7} counts {c7/c*100}%")
# print(s8 + f": {c8} counts {c8/c*100}%")
# print(s3 + f": {c3} counts {c3/c*100}%")
# print(s4 + f": {c4} counts {c4/c*100}%")
# print(s5 + f": {c5} counts {c5/c*100}%")
# print(s6 + f": {c6} counts {c6/c*100}%")

In [3]:
hist_dict = stage0.read_rawroot_to_dict("../data/rawroot_run_0014*")

## Write Histograms
Write histograms from dictionary to a root file maintaing heirerachy. 

In [ ]:
det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,19,-1,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] # det number for index of position number runs 13~54
# det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,-1,19,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] #det number for index of position number runs 55~90


det_pos_map = ROOT.TH2I("det_pos_map","det_pos_map;position # ;detector # ",32,0,32,32,0,32)
det_pos_graph = ROOT.TGraph()
det_pos_graph.SetTitle("det_pos_graph")
det_pos_graph.SetName("det_pos_graph")
det_pos_graph.GetXaxis().SetTitle("positon #")
det_pos_graph.GetYaxis().SetTitle("detector #")



for ch in range(numch):
    det_pos_map.SetBinContent(ch+1,det_number[ch]+1,1)
    det_pos_graph.AddPoint(ch,det_number[ch])

hist_dict[det_pos_map.GetName()] = det_pos_map
hist_dict[det_pos_graph.GetName()] = det_pos_graph



In [ ]:
utils.write_dict_to_root(hist_dict, "stage0_output.root")